In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from database_io.db_handler import DB_handler
import pymc as pm
import numpy as np
import pandas as pd

import plotly.express as px

[01/03/25 11:07:32] INFO     Custom team name replacements loaded from                                ]8;id=959209;file:///home/morten/.conda/envs/most_pymc/lib/python3.12/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=599433;file:///home/morten/.conda/envs/most_pymc/lib/python3.12/site-packages/soccerdata/_config.py#82\82]8;;\
                             /home/morten/soccerdata/config/teamname_replacements.json.                            

                    INFO     Custom league dict loaded from                                          ]8;id=875859;file:///home/morten/.conda/envs/most_pymc/lib/python3.12/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=469529;file:///home/morten/.conda/envs/most_pymc/lib/python3.12/site-packages/soccerdata/_config.py#160\160]8;;\
                             /home/morten/soccerdata/config/league_dict.json.                                      

In [3]:
db = DB_handler()

In [4]:
games = db.games.get_all_games(0.1)

In [5]:
# games["minutes_missed"] = (90 - games["minutes"]).clip(lower=0)
games["elo_diff"] = games["elo"] - games["opposition_elo"]
games["goal_diff"] = games["result"].apply(lambda x: int(x.split("-")[0]) - int(x.split("-")[1]))

In [6]:
games["elo_diff"] = games["elo_diff"] / (elo_std:=max(games["elo_diff"].max(), abs(games["elo_diff"].min())))
# games["goal_diff"] = games["goal_diff"] / (goal_std:=max(games["goal_diff"].max(), abs(games["goal_diff"].min())))
games["minutes_missed"] = (games["game_minutes"] - games["minutes"]).clip(lower=0)

# games["minutes_missed"] = games["minutes_missed"] / (min_std:=games["minutes_missed"].max())

In [7]:
games["minutes_missed"] = (games["minutes_missed"] / (minutes_missed_std:=max(games["minutes_missed"].max(), abs(games["minutes_missed"].min()))))


In [8]:
games["minutes_missed_abs"] = games["minutes_missed"].abs()
games["goal_diff_abs"] = games["goal_diff"].abs()

In [9]:
px.scatter(games, "goal_diff_abs", "minutes_missed_abs")

In [10]:
print(elo_std, minutes_missed_std)

310.4568814848583 103


In [11]:
games

,id,minutes,elo,opposition_elo,result,home,game_minutes,elo_diff,goal_diff,minutes_missed,minutes_missed_abs,goal_diff_abs
0,1643586,10,1350.989041,1411.457147,0-0,0,97,-0.194771,0,0.844660,0.844660,0
1,1643586,10,1387.795954,1411.457147,0-0,0,97,-0.076214,0,0.844660,0.844660,0
2,1643586,10,1359.833623,1411.457147,0-0,0,97,-0.166282,0,0.844660,0.844660,0
3,1643297,25,1481.762428,1467.373481,0-0,0,98,0.046348,0,0.708738,0.708738,0
4,1643297,24,1492.921067,1467.523141,0-0,0,98,0.081808,0,0.718447,0.718447,0
...,...,...,...,...,...,...,...,...,...,...,...,...
69351,1388613,35,1351.351537,1388.682000,2-0,1,94,-0.120244,2,0.572816,0.572816,2
69352,1388613,24,1385.301427,1387.035839,2-0,1,94,-0.005587,2,0.679612,0.679612,2
69353,1388613,24,1348.313831,1387.035839,2-0,1,94,-0.124726,2,0.679612,0.679612,2
69354,1388613,21,1341.184189,1372.855581,0-2,0,94,-0.102015,-2,0.708738,0.708738,2


In [12]:
with pm.Model() as MoV:
    # mutable data
    is_home = games["home"].values
    elo_diff = games["elo_diff"].values
    minutes_missed = games["minutes_missed"].values
    goal_diff = games["goal_diff"].values

    # pymc vars
    home_advantage = pm.Normal("home_advantage", mu = 0.3, sigma = 1)
    elo_diff_coeff = pm.Normal("elo_diff_coeff", mu = 1, sigma = 1)
    minutes_missed_coeff = pm.Normal("minutes_missed_coeff", mu = 1, sigma = 1)
    mean_ = pm.Deterministic("mean_", ((is_home * home_advantage) + (elo_diff * elo_diff_coeff)) * pm.math.abs(1 - (minutes_missed * minutes_missed_coeff))) # instead of abs -> if < 0 then 0 ?
    var_ = pm.HalfNormal("var_")
    goal_diff = pm.Normal("goal_diff", mean_, var_, observed=goal_diff)

In [ ]:
with MoV:
    trace = pm.sample(10000)

[01/03/25 11:08:03] INFO     Auto-assigning NUTS sampler...                                             ]8;id=929823;file:///home/morten/.conda/envs/most_pymc/lib/python3.12/site-packages/pymc/sampling/mcmc.py\mcmc.py]8;;\:]8;id=894503;file:///home/morten/.conda/envs/most_pymc/lib/python3.12/site-packages/pymc/sampling/mcmc.py#748\748]8;;\

                    INFO     Initializing NUTS using jitter+adapt_diag...                              ]8;id=28052;file:///home/morten/.conda/envs/most_pymc/lib/python3.12/site-packages/pymc/sampling/mcmc.py\mcmc.py]8;;\:]8;id=251562;file:///home/morten/.conda/envs/most_pymc/lib/python3.12/site-packages/pymc/sampling/mcmc.py#1424\1424]8;;\

[01/03/25 11:08:07] INFO     Multiprocess sampling (4 chains in 4 jobs)                                 ]8;id=852101;file:///home/morten/.conda/envs/most_pymc/lib/python3.12/site-packages/pymc/sampling/mcmc.py\mcmc.py]8;;\:]8;id=87643;file:///home/morten/.conda/envs/most_pymc/lib/python3.12/site-packages/pymc/sampling/mcmc.py#843\843]8;;\

                    INFO     NUTS:                                                                      ]8;id=570444;file:///home/morten/.conda/envs/most_pymc/lib/python3.12/site-packages/pymc/sampling/mcmc.py\mcmc.py]8;;\:]8;id=707398;file:///home/morten/.conda/envs/most_pymc/lib/python3.12/site-packages/pymc/sampling/mcmc.py#252\252]8;;\

Output()

In [18]:
# (is_home * home_advantage) + (elo_diff * elo_diff_coeff) - (minutes_missed * minutes_missed_coeff))
home_test = 0
elo_diff_test = 70 / elo_std
minutes_missed_test = 45 / minutes_missed_std

(home_test * trace.posterior.home_advantage.values[0][0] + elo_diff_test * trace.posterior.elo_diff_coeff.values[0][0]) * abs(1 -  minutes_missed_test * trace.posterior.minutes_missed_coeff.values[0][0])

0.3811707658117415

In [19]:
trace.to_netcdf("/home/morten/Develop/packing-report/metrics/mov_elo/mov_elo.nc")

'/home/morten/Develop/packing-report/metrics/mov_elo/mov_elo.nc'